In [7]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import pickle
import re
import collections
import tables as tb
import ipyparallel as ipp
from sklearn.cluster import spectral_clustering 
import scipy as sp
from sklearn.feature_extraction import image
import time

In [8]:
lena = sp.misc.lena()
# Downsample the image by a factor of 4
lena = lena[::2, ::2] + lena[1::2, ::2] + lena[::2, 1::2] + lena[1::2, 1::2]
lena = lena[::2, ::2] + lena[1::2, ::2] + lena[::2, 1::2] + lena[1::2, 1::2]

# Convert the image into a graph with the value of the gradient on the
# edges.
graph = image.img_to_graph(lena)

# Take a decreasing function of the gradient: an exponential
# The smaller beta is, the more independent the segmentation is of the
# actual image. For beta=1, the segmentation is close to a voronoi
beta = 5
eps = 1e-6
graph.data = np.exp(-beta * graph.data / lena.std()) + eps

# Apply spectral clustering (this step goes much faster if you have pyamg
# installed)
N_REGIONS = 11

###############################################################################
# Visualize the resulting regions

for assign_labels in ('kmeans', 'discretize'):
    t0 = time.time()
    labels = spectral_clustering(graph, n_clusters=N_REGIONS,
                                 assign_labels=assign_labels,
                                 random_state=1)
    t1 = time.time()
    labels = labels.reshape(lena.shape)

    plt.figure(figsize=(5, 5))
    plt.imshow(lena,   cmap=plt.cm.gray)
    for l in range(N_REGIONS):
        plt.contour(labels == l, contours=1,
                    colors=[plt.cm.spectral(l / float(N_REGIONS)), ])
    plt.xticks(())
    plt.yticks(())
    plt.title('Spectral clustering: %s, %.2fs' % (assign_labels, (t1 - t0)))

plt.show()

/home/nikhil/.conda/envs/adni/lib/python2.7/site-packages/matplotlib/collections.py:650: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != str('face'):
/home/nikhil/.conda/envs/adni/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [32]:
N = 100
nodes = np.arange(N)
b = np.random.rand(N,N)
b_symm = (b + b.T)/2

In [45]:
assign_labels = 'kmeans'
labels = spectral_clustering(b_symm, n_clusters=N_REGIONS,
                                 =assign_labels,
                                 random_state=1)

In [46]:
labels

array([ 1, 10,  2,  8,  4,  9,  0,  1,  9,  0,  0,  1,  8, 10,  8,  3,  8,
        2,  0,  4,  8,  4,  2,  2,  9,  7,  0,  2,  1,  5,  2,  5,  8,  8,
        3,  2,  8,  4,  2,  7, 10,  9,  2,  4,  6,  0,  7,  0,  2,  5,  3,
        3,  9,  5,  7,  7,  0,  1,  2,  4,  6,  5,  3,  7,  9,  2, 10,  3,
        1,  2,  1,  5,  2,  3,  7, 10,  3,  2,  0,  6,  9,  7,  5,  6,  9,
        9,  4,  5,  4,  7,  2,  6,  3,  7,  5,  8,  0,  8,  1,  5], dtype=int32)